In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"
# os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'upb'

In [ ]:
os.environ['TRANSFORMERS_CACHE'] = './models'
print(f"Current cache directory: {os.environ['TRANSFORMERS_CACHE']}")

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
from transformers import pipeline

device = "cuda"

In [4]:
model = "OLMo-7B-0724-Instruct-hf"

In [5]:
pipe = pipeline("text-generation", model="allenai/{}".format(model), max_length=2048, device=device)

config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/18.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.83G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/5.73k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

test stage

In [6]:
import json
from tqdm import tqdm

In [7]:
def convert_unicode_to_plain_text(unicode_str):
    if isinstance(unicode_str, str) and '\\u' in unicode_str:
        plain_text = unicode_str.encode().decode('unicode_escape')
        return plain_text
    else:
        return unicode_str

In [8]:
template = "Give me a sense of your preference regarding the following question or statement. Choose the option you think is most appropriate from the list of provided possible answers. \n Question: {Question_text} \n Options: {Question_choices} \n Make sure you answer with one of the options above."

In [9]:
para_dict = [{
    "role": "user",
    "content": ""
    }]

para_dict2 = [
    {
        "role": "user",
        "content": "Some people have a political issue that they care about more than most other issues. They might think about the issue a lot. They might pay particular attention to news about that issue, even when it's not making national news. They might focus on what political candidates say about that issue, and decide who to vote for on the basis of that issue. Or they might just care about the issue a lot. Is there an issue like that for you?"
    },
    {"role": "assistant",
        "content": "Yes"}, 
    {"role": "user",
        "content":  ""}
    ]

In [ ]:
root = "./question_json_ver3"
script_list = os.listdir(root)
script_list.sort()

In [ ]:
if not os.path.exists(model): os.mkdir(model)

for script in tqdm(script_list):
    if not os.path.exists(os.path.join(model, script)):
# if 1:
#         script = script_list[0]
        print("working on {} ...".format(script))

        with open(os.path.join(root, script), 'r') as f:
            data = json.load(f)
        for i, question in tqdm(enumerate(data['questions'])):
            execute = 0
            if "Answer_text" not in question.keys():
                q, c = question["Question_text"], question["Question_choices"]
                if len(c) > 0:
                    execute = 1
                    template_tmp = template.format(**({"Question_text":q, "Question_choices":c}))

                    para_tmp = para_dict.copy()
                    para_tmp[-1]["content"] = template_tmp
                elif len(c) == 0 and ("yes" in data['questions'][i-1]["Answer_text"].lower()):
                    execute = 1
                    template_tmp = q

                    para_tmp = para_dict2.copy()
                    para_tmp[-1]["content"] = template_tmp
                else:
                    continue

                if execute:
                    try:
                            response = pipe(para_tmp)
                            output = response[-1]['generated_text'][-1]["content"] 
                    except:
                            response = "#ERROR"
                    data['questions'][i]["Answer_text"] = convert_unicode_to_plain_text(output)
                    
                with open(os.path.join(model, script), 'w', encoding='utf-8') as f:
                    json.dump(data, f, indent=4)
            

  0%|          | 0/10 [00:00<?, ?it/s]

working on questions_json_1.json ...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
45it [01:29,  1.99s/it]
 10%|█         | 1/10 [01:29<13:26, 89.58s/it]

working on questions_json_10.json ...


45it [01:25,  1.90s/it]
 20%|██        | 2/10 [02:54<11:36, 87.11s/it]

...

45it [01:30,  2.01s/it]
 80%|████████  | 8/10 [11:38<02:54, 87.43s/it]

working on questions_json_8.json ...


45it [01:22,  1.83s/it]
 90%|█████████ | 9/10 [13:01<01:25, 85.82s/it]

working on questions_json_9.json ...


45it [01:29,  1.98s/it]
100%|██████████| 10/10 [14:30<00:00, 87.01s/it]
